# You Tube transcripts

In [1]:
from utils import Crawler, Analyzer, aggregate_analysis_files, VideoInfo

%load_ext autoreload
%autoreload 2

# Search for YT channels of Florida cities and towns

So we go over the list of all cities in Florida and search YouTube for "city of XYZ Florida" and "town of XYZ Florida". This is what `Crawler` class does. See docstring in Crawler for details.

In [2]:
crawler = Crawler()

In [3]:
crawler.start()

252it [00:33,  7.55it/s] 

Quota exceeded. Stopping crawling.


For each json response we now ask ChatGPT to determine is the channel official or not, this creates csv file in analysis folder, and updates in status.csv: 

In [8]:
analyzer = Analyzer()

In [9]:
analyzer.start()

437it [00:15, 29.02it/s]


We now aggregate the results in a single csv file, storing only yes results:

In [4]:
import os
import pandas as pd
from pandas.errors import EmptyDataError
from utils import ANALYSIS_DIR


In [3]:
aggregate_analysis_files(crawler, 'aggregated_analysis.csv')

Error reading analysis/AT00007244.csv. Skipping it.


## Extract info from one video with video id

In [3]:
# Example YouTube video ID
# https://www.youtube.com/watch?v=AH4XSLg6rvw&t=588s
video_id = "5AIBek1jhfE"  # ID is what comes after 'v=' in URL: https://www.youtube.com/watch?v=5AIBek1jhfE

In [5]:
video = VideoInfo(video_id)

In [24]:
import os
from googleapiclient.discovery import build


# Initialize the YouTube Data API client
youtube = build('youtube', 'v3', developerKey=yt_api_key)


def get_channel_id(youtube, channel_username):
    request = youtube.channels().list(
        part="id",
        forUsername=channel_username
    )
    response = request.execute()
    return response['items'][0]['id'] if response['items'] else None


def get_live_streams_from_channel(channel_id):
    videos = []
    next_page_token = None

    while True:
        request = youtube.search().list(
            part="snippet",
            channelId=channel_id,
            maxResults=50,
            pageToken=next_page_token,
            order="date",
            type="video",
            eventType="live"  # This filters for live streams only
        )
        response = request.execute()

        for item in response['items']:
            video_info = {
                "video_id": item['id']['videoId'],
                "title": item['snippet']['title'],
                "description": item['snippet']['description'],
                "published_at": item['snippet']['publishedAt']
            }
            videos.append(video_info)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return videos


In [ ]:

# Example channel username (can also be channel ID if known)
channel_username = "townoffortmyersbeachfl33931"

# Get the channel ID
channel_id = get_channel_id(youtube, channel_username)
if not channel_id:
    print(f"Channel not found for username: {channel_username}")
else:
    # Get the live streams from the channel
    live_streams = get_live_streams_from_channel(channel_id)
    
    # Print the live streams
    for video in live_streams:
        print(f"Video ID: {video['video_id']}")
        print(f"Title: {video['title']}")
        print(f"Description: {video['description']}")
        print(f"Published At: {video['published_at']}")
        print("-" * 40)